In [8]:
import numpy as np

#load the original coordinates
xyz_orig = np.loadtxt("P_0.vasp", skiprows=8)

#load the structure with the displacements of the tetragonal rotational transition mode
dxyz_orig = np.loadtxt("P_disp.vasp", skiprows=8)

#corresponding amplitudes in Angstrom for the tetragonal mode
a_orig = 1


a_min = 0
a_max = 1.2
a_fac = -1

ndisp = 40

#POSCAR header
poscar_header = """
1.0
        5.177959919             0.0             0.0
        0.0             5.177959919             0.0
        0.0             0.0             7.7496299744
    W    O
    4   12
Direct
"""

#calculate the displacement
dxyz = xyz_orig - dxyz_orig

for idisp in range(1, ndisp+1):
    a = a_min + idisp* (a_max - a_min)/ndisp 
    new_xyz = xyz_orig + a/a_orig*dxyz * a_fac

    #make the poscar file
    poscar_file = poscar_header
    for j in range(0, xyz_orig.shape[0]):
        for k in range(0, 3):
            poscar_file += str(new_xyz[j,k]) + '\t'
        poscar_file += '\n'
    
    new_file = "P_" + str(idisp) + ".vasp"
    new_file = open(new_file, 'w')
    new_file.write(poscar_file)
    new_file.close()


In [17]:
import numpy as np

#load the original coordinates where tetragonal P order parameter is 0
xyz_orig_tetr = np.loadtxt("P_0_low.vasp", skiprows=8)

#load the original coordinates where orthorhombic Q order parameter is 0
#note that P_0 and Q_0 structures should be equivalent apart from atomic indices permutations
xyz_orig_orth = np.loadtxt("Q_0.vasp", skiprows=8)

#load the structure with the displacements of the tetragonal rotational transition mode
dxyz_orig_tetr = np.loadtxt("P_disp_low.vasp", skiprows=8)

#corresponding amplitude in Angstrom for the tetragonal mode P in dxyz_orig_tetr
a_orig = 1

#amplitude range for the tetragonal mode P
a_min = 0
a_max = 1.2
a_fac = -1

#number of mode increments
ndisp = 40

#permutation matrix of corresponding atoms after generating tetragonal rotaional mode displacements
rot_index = np.empty(32)
rot_index[0]=0
rot_index[1]=1
rot_index[2]=3
rot_index[3]=7
rot_index[4]=6
rot_index[5]=2
rot_index[6]=5
rot_index[7]=4
rot_index[8]=8
rot_index[9]=9
rot_index[10]=11
rot_index[11]=15
rot_index[12]=14
rot_index[13]=10
rot_index[14]=13
rot_index[15]=12
rot_index[16]=24
rot_index[17]=25
rot_index[18]=16
rot_index[19]=17
rot_index[20]=18
rot_index[21]=19
rot_index[22]=26
rot_index[23]=27
rot_index[24]=28
rot_index[25]=21
rot_index[26]=30
rot_index[27]=23
rot_index[28]=22
rot_index[29]=31
rot_index[30]=20
rot_index[31]=29

#POSCAR header
poscar_header = """
1.0
        7.3236460686 0 0
        0 7.7495899200 0
        0 0 7.3236460686
    W    O
    8   24
Direct
"""

#check if there are doubles in the permutation array
for i in range(0, rot_index.size):
    comp = rot_index[i]
    for j in range(0, rot_index.size):
        if not i == j and comp == rot_index[j]:
            print("doubles in permutation matrix")
            
#construct the permutation matrix
perm_matrix = np.empty((rot_index.size, rot_index.size))
for i in range(0, rot_index.size):
    for j in range(0, rot_index.size):
        if j == rot_index[i]:
            perm_matrix[i,j] = 1
        else:
            perm_matrix[i,j] = 0

for idisp in range(1, ndisp+1):
    #load the structure containing the displacements of the orthorhombic rotational transition mode Q
    dxyz_orth = np.loadtxt("Q_" + str(idisp) + ".vasp", skiprows=8)
    
    #get the displacement for the tetragonal rotational transition mode P
    a = a_min + idisp* (a_max - a_min)/ndisp 
    dxyz_tetr = (dxyz_orig_tetr - xyz_orig_tetr) * a/a_orig 
    
    #permutate
    dxyz_tetr = perm_matrix.dot(dxyz_tetr)
    
    #add the tetragonal mode displacement P to the structure already containing the orthorhombic mode Q    
    new_xyz = dxyz_orth + dxyz_tetr
    
    #make the poscar file
    poscar_file = poscar_header
    for j in range(0, new_xyz.shape[0]):
        for k in range(0, 3):
            poscar_file += str(new_xyz[j,k]) + '\t'
        poscar_file += '\n'
    
    new_file = "PQ_" + str(idisp) + ".vasp"
    new_file = open(new_file, 'w')
    new_file.write(poscar_file)
    new_file.close()
    